In [5]:
from google.colab import files
import os, pandas as pd
from sqlalchemy import create_engine, inspect
from pathlib import Path

DATA_DIR = "/content/olist_manual"
DB_PATH  = "/content/olist.sqlite"
os.makedirs(DATA_DIR, exist_ok=True)

# Upload all 9 CSVs in one go
uploads = files.upload()  # multi-select the 9 CSVs on your computer
for name, data in uploads.items():
    with open(os.path.join(DATA_DIR, name), "wb") as f:
        f.write(data)

# Verify
required = [
    "olist_customers_dataset.csv","olist_sellers_dataset.csv","olist_order_reviews_dataset.csv",
    "olist_order_items_dataset.csv","olist_products_dataset.csv","olist_geolocation_dataset.csv",
    "product_category_name_translation.csv","olist_orders_dataset.csv","olist_order_payments_dataset.csv",
]
missing = [fn for fn in required if not (Path(DATA_DIR)/fn).exists()]
assert not missing, f"Missing files: {missing}"

# Create SQLite
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)
mapping = {
    "olist_customers_dataset.csv": "olist_customers",
    "olist_sellers_dataset.csv": "olist_sellers",
    "olist_order_reviews_dataset.csv": "olist_order_reviews",
    "olist_order_items_dataset.csv": "olist_order_items",
    "olist_products_dataset.csv": "olist_products",
    "olist_geolocation_dataset.csv": "olist_geolocation",
    "product_category_name_translation.csv": "product_category_name_translation",
    "olist_orders_dataset.csv": "olist_orders",
    "olist_order_payments_dataset.csv": "olist_order_payments",
}
for fname, tname in mapping.items():
    pd.read_csv(Path(DATA_DIR)/fname).to_sql(tname, con=engine, if_exists="replace", index=False)

print("Tables:", inspect(engine).get_table_names())
pd.read_sql_query("SELECT * FROM olist_customers LIMIT 5;", con=engine)


Saving olist_customers_dataset.csv to olist_customers_dataset (1).csv
Saving olist_geolocation_dataset.csv to olist_geolocation_dataset (1).csv
Saving olist_order_items_dataset.csv to olist_order_items_dataset (1).csv
Saving olist_order_payments_dataset.csv to olist_order_payments_dataset (1).csv
Saving olist_order_reviews_dataset.csv to olist_order_reviews_dataset (1).csv
Saving olist_orders_dataset.csv to olist_orders_dataset (1).csv
Saving olist_products_dataset.csv to olist_products_dataset (1).csv
Saving olist_sellers_dataset.csv to olist_sellers_dataset (2).csv
Saving product_category_name_translation.csv to product_category_name_translation (2).csv
Tables: ['olist_customers', 'olist_geolocation', 'olist_order_items', 'olist_order_payments', 'olist_order_reviews', 'olist_orders', 'olist_products', 'olist_sellers', 'product_category_name_translation']


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [9]:
q1 = """
SELECT
    ROUND(
        (SELECT COUNT(*) FROM olist_orders AS od
         JOIN olist_order_reviews AS ord ON ord.order_id = od.order_id
         WHERE od.order_purchase_timestamp LIKE '2018-01%%'
         AND ord.review_score = 5) * 100.0 /
        (SELECT COUNT(*) FROM olist_orders
         WHERE order_purchase_timestamp LIKE '2018-01%%'),
    2) AS percentage;
"""
df_q1 = pd.read_sql_query(q1, engine)
df_q1


,percentage
0,56.36


In [10]:
q2 = """
WITH yearly_stats AS (
    SELECT
        STRFTIME('%Y', ord.order_purchase_timestamp) AS year,
        COUNT(DISTINCT ord.order_id) AS total_orders, -- Используем DISTINCT на случай дублей
        COUNT(DISTINCT cust.customer_unique_id) AS unique_customers
    FROM olist_orders AS ord
    INNER JOIN olist_customers AS cust
        ON cust.customer_id = ord.customer_id
    GROUP BY year
)
SELECT
    year,
    total_orders,
    unique_customers,
    ROUND(100.0 * (total_orders - LAG(total_orders) OVER (ORDER BY year))
        / NULLIF(LAG(total_orders) OVER (ORDER BY year), 0), 2) AS orders_growth_pct,
    ROUND(100.0 * (unique_customers - LAG(unique_customers) OVER (ORDER BY year))
        / NULLIF(LAG(unique_customers) OVER (ORDER BY year), 0), 2) AS customers_growth_pct
FROM yearly_stats
ORDER BY year;
"""
df_q2 = pd.read_sql_query(q2, engine)
df_q2


,year,total_orders,unique_customers,orders_growth_pct,customers_growth_pct
0,2016,329,326,NaN,NaN
1,2017,45101,43713,13608.51,13308.90
2,2018,54011,52749,19.76,20.67


In [16]:
q3 = '''
SELECT
    cust.customer_unique_id,
    strftime('%Y-%m', ord.order_purchase_timestamp) AS year_month, -- Обрати внимание на '%Y-%m'
    ROUND(SUM(opay.payment_value) * 1.0 / COUNT(DISTINCT ord.order_id), 2) AS average_order_value
FROM olist_customers AS cust
INNER JOIN olist_orders AS ord ON ord.customer_id = cust.customer_id
INNER JOIN olist_order_payments AS opay ON opay.order_id = ord.order_id
GROUP BY cust.customer_unique_id, year_month
ORDER BY year_month DESC, average_order_value DESC
'''

df_q3 = pd.read_sql_query(q3, con=engine)
print(df_q3.head(20))

                  customer_unique_id year_month  average_order_value
0   262e1f1e26e92e86375f86840b4ffd63    2018-10               222.03
1   af5454198a97379394cacf676e1e96cb    2018-10               197.55
2   87ab9fec999db8bd5774917de3cdf01c    2018-10                89.71
3   634420a0ea42302205032ed44ac7fccc    2018-10                80.38
4   0a5dadb73145d29f42f703c97c377ae8    2018-09              2266.61
5   c1ee153508c6b785b491443a95ff364e    2018-09               414.04
6   09687a7b7431a93b5c53b50ba779bf94    2018-09               258.67
7   9c3af16efacb7aa06aa3bc674556c5d6    2018-09               221.54
8   af5454198a97379394cacf676e1e96cb    2018-09               197.55
9   08642cd329066fe11ec63293f714f2f8    2018-09               191.46
10  ff22e30958c13ffe219db7d711e8f564    2018-09               166.46
11  9bb92bebd4cb7511e1a02d5e50bc4655    2018-09               137.03
12  d05c44a138277ad325d915c6b7ccbcdf    2018-09               107.13
13  ba84da8c159659f116329563a0a981

In [19]:
q4 = """
SELECT
    cust.customer_city,
    ROUND(SUM(opay.payment_value), 2) AS revenue
FROM olist_order_payments AS opay
INNER JOIN olist_orders AS ord
    ON ord.order_id = opay.order_id
INNER JOIN olist_customers AS cust
    ON cust.customer_id = ord.customer_id
WHERE ord.order_purchase_timestamp BETWEEN '2016-01-01' AND '2018-12-31'
GROUP BY cust.customer_city
ORDER BY revenue DESC
LIMIT 5;
"""
df_q4 = pd.read_sql_query(q4, engine)
df_q4


,customer_city,revenue
0,sao paulo,2203373.09
1,rio de janeiro,1161927.36
2,belo horizonte,421765.12
3,brasilia,354216.78
4,curitiba,247392.48


In [21]:
q5 = """
SELECT
    cust.customer_state,
    ROUND(SUM(opay.payment_value), 2) AS revenue
FROM olist_order_payments AS opay
INNER JOIN olist_orders AS ord
    ON ord.order_id = opay.order_id
INNER JOIN olist_customers AS cust
    ON cust.customer_id = ord.customer_id
WHERE ord.order_purchase_timestamp BETWEEN '2016-01-01' AND '2018-12-31'
GROUP BY cust.customer_state
ORDER BY revenue DESC;
"""
df_q5 = pd.read_sql_query(q5, engine)
print(df_q5.head(20))

   customer_state     revenue
0              SP  5998226.96
1              RJ  2144379.69
2              MG  1872257.26
3              RS   890898.54
4              PR   811156.38
5              SC   623086.43
6              BA   616645.82
7              DF   355141.08
8              GO   350092.31
9              ES   325967.55
10             PE   324850.44
11             CE   279464.03
12             PA   218295.85
13             MT   187029.29
14             MA   152523.02
15             PB   141545.72
16             MS   137534.84
17             PI   108523.97
18             RN   102718.13
19             AL    96962.06


In [23]:
q6 = """
SELECT
    oi.seller_id,
    COUNT(oi.order_id) AS total_goods_sold,
    ROUND(SUM(oi.price), 2) AS total_revenue,
    COUNT(DISTINCT ord.customer_id) AS total_customers,
    SUM(CASE WHEN orev.review_score = 5 THEN 1 ELSE 0 END) AS count_5_star_ratings,
    ROUND(AVG(orev.review_score), 2) AS avg_rating
FROM olist_order_items AS oi
INNER JOIN olist_orders AS ord
    ON oi.order_id = ord.order_id
INNER JOIN olist_order_reviews AS orev
    ON oi.order_id = orev.order_id
GROUP BY oi.seller_id
HAVING count_5_star_ratings > 50
ORDER BY total_revenue DESC
LIMIT 10;
"""
df_q6 = pd.read_sql_query(q6, engine)
df_q6

,seller_id,total_goods_sold,total_revenue,total_customers,count_5_star_ratings,avg_rating
0,4869f7a5dfa277a7dca6462dcf3b52b2,1148,228071.04,1124,683,4.12
1,53243585a1d6dc2643021fd1853d8905,408,220740.05,356,210,4.08
2,4a3ca9315b744ce9f8e9374361493884,1984,200561.42,1785,947,3.80
3,fa1c13f2614d7b5c4749cbc52fecda94,582,192774.43,581,394,4.34
4,7c67e1448b00f6e969d365cea6b010ab,1367,188017.85,976,437,3.35
5,7e93a43ef30c4f03f38b393420bc753a,339,176201.88,335,213,4.21
6,da8622b14eb17ae2831f4ac5b9dab84a,1568,161993.97,1308,893,4.07
7,7a67c85e85bb2ce8582c35f2203ad736,1166,141130.58,1151,717,4.23
8,1025f0e2d44d7041d6cf58b6550e0bfa,1431,139484.38,907,729,3.85
9,955fee9216a65b617aa5c0531780ce60,1489,133948.81,1277,804,4.05


In [25]:
q7 = """
SELECT
    cust.customer_state,
    COUNT(ord.order_id) AS total_orders_in_state,
    SUM(CASE WHEN ord.order_status = 'delivered' THEN 1 ELSE 0 END) AS delivered_in_state,
    ROUND(100.0 * SUM(CASE WHEN ord.order_status = 'delivered' THEN 1 ELSE 0 END) / COUNT(ord.order_id), 2) AS delivery_success_rate
FROM olist_customers AS cust
INNER JOIN olist_orders AS ord
    ON ord.customer_id = cust.customer_id
GROUP BY cust.customer_state
ORDER BY delivery_success_rate DESC;
"""
df_q7 = pd.read_sql_query(q7, engine)
df_q7

,customer_state,total_orders_in_state,delivered_in_state,delivery_success_rate
0,AC,81,80,98.77
1,AP,68,67,98.53
2,ES,2033,1995,98.13
3,MS,715,701,98.04
4,AM,148,145,97.97
5,TO,280,274,97.86
6,RS,5466,5345,97.79
7,RN,485,474,97.73
8,MT,907,886,97.68
9,PR,5045,4923,97.58


In [28]:
q8 = """
SELECT
    prod.product_category_name,
    opay.payment_type,
    COUNT(opay.payment_type) AS payments_for_products
FROM olist_order_payments AS opay
INNER JOIN olist_order_items AS oi
    ON oi.order_id = opay.order_id
INNER JOIN olist_products AS prod
    ON prod.product_id = oi.product_id
WHERE prod.product_category_name IS NOT NULL
GROUP BY prod.product_category_name, opay.payment_type
ORDER BY payments_for_products DESC;
"""
df_q8 = pd.read_sql_query(q8, engine)
print(df_q8.head(10))

    product_category_name payment_type  payments_for_products
0         cama_mesa_banho  credit_card                   8959
1            beleza_saude  credit_card                   7566
2           esporte_lazer  credit_card                   6635
3        moveis_decoracao  credit_card                   6379
4  informatica_acessorios  credit_card                   5436
5   utilidades_domesticas  credit_card                   5411
6      relogios_presentes  credit_card                   4858
7               telefonia  credit_card                   3400
8              brinquedos  credit_card                   3294
9              automotivo  credit_card                   3277


In [31]:
q9 = """
WITH city_coords AS (
    SELECT
        lower(replace(replace(replace(replace(replace(replace(geolocation_city,
            'á', 'a'), 'é', 'e'), 'í', 'i'), 'ó', 'o'), 'ú', 'u'), 'ê', 'e')) AS clean_city,
        AVG(geolocation_lat) AS lat,
        AVG(geolocation_lng) AS lng
    FROM olist_geolocation
    GROUP BY clean_city
)
SELECT
    ord.order_id,
    cust.customer_city,
    sel.seller_city,
    ROUND(6371 * ACOS(
        COS(c_coord.lat * 0.01745) * COS(s_coord.lat * 0.01745) * COS((s_coord.lng - c_coord.lng) * 0.01745) +
        SIN(c_coord.lat * 0.01745) * SIN(s_coord.lat * 0.01745)
    ), 2) AS distance_km
FROM olist_orders AS ord
INNER JOIN olist_customers AS cust ON ord.customer_id = cust.customer_id
INNER JOIN olist_order_items AS itm ON ord.order_id = itm.order_id
INNER JOIN olist_sellers AS sel ON itm.seller_id = sel.seller_id
INNER JOIN city_coords AS c_coord ON
    lower(replace(replace(replace(replace(replace(replace(cust.customer_city, 'á', 'a'), 'é', 'e'), 'í', 'i'), 'ó', 'o'), 'ú', 'u'), 'ê', 'e')) = c_coord.clean_city
INNER JOIN city_coords AS s_coord ON
    lower(replace(replace(replace(replace(replace(replace(sel.seller_city, 'á', 'a'), 'é', 'e'), 'í', 'i'), 'ó', 'o'), 'ú', 'u'), 'ê', 'e')) = s_coord.clean_city
LIMIT 10;
"""
df_q9 = pd.read_sql_query(q9, engine)
df_q9

,order_id,customer_city,seller_city,distance_km
0,e481f51cbdc54678b7cc49136f2d6af7,sao paulo,maua,21.77
1,53cdb2fc8bc7dce0b6741e2150273451,barreiras,belo horizonte,869.47
2,47770eb9100c2d0c44946d9cf07ec65d,vianopolis,guariba,514.25
3,949d5b44dbf5de918fe9c16f97b45f8a,sao goncalo do amarante,belo horizonte,1841.44
4,ad21c59c0840e6cb83a9ceb5573f8159,santo andre,mogi das cruzes,36.00
5,a4591c265e18cb1dcee52889e2d8acc3,congonhinhas,guarulhos,413.20
6,136cce7faa42fdb2cefd53fdc79a6098,santa rosa,sao paulo,919.08
7,6514b8ad8028c9f2cc2374ded245783f,nilopolis,atibaia,323.55
8,76c6e866289321a7c93b82b54852dc33,faxinalzinho,sao jose do rio pardo,870.01
9,e69bfb5eb88e0ed6a785585b27e16dbf,sorocaba,itaquaquecetuba,114.39
